<a href="https://colab.research.google.com/github/thanhdaibl1572002/data-mining-project/blob/main/DATA_MINING_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GIỚI THIỆU**
###**- Đề tài:** Dự đoán Tính Khả Dụng Tín Dụng Dựa trên Thông Tin Cá Nhân Khách Hàng.
###**- Phương pháp:** Random Forest.
###**- Nhãn dự đoán:** "Bad" hoặc "Good"
###**- Bộ dữ liệu:**: [Credit Card Approval Prediction](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction)



#**IMPORT THƯ VIỆN**

In [ ]:
import pandas as pd

#**TẢI BỘ DỮ LIỆU**

In [ ]:
! git clone 'https://github.com/thanhdaibl1572002/data-mining-project'

#**TỔNG QUAN DỮ LIỆU**

**- ID**: Mã số khách hàng

**- CODE_GENDER**: Giới tính

**- FLAG_OWN_CAR**: Có xe hơi không

**- FLAG_OWN_REALTY**: Có tài sản bất động sản không

**- CNT_CHILDREN**: Số lượng con cái

**- AMT_INCOME_TOTAL**: Thu nhập hàng năm

**- NAME_INCOME_TYPE**: Hạng mục thu nhập

**- NAME_EDUCATION_TYPE**: Cấp độ giáo dục

**- NAME_FAMILY_STATUS**: Tình trạng hôn nhân

**- NAME_HOUSING_TYPE**: Cách sống

**- DAYS_BIRTH**: Ngày sinh (Số ngày tính ngược từ ngày hiện tại, 0 là hôm nay, -1 là hôm qua)

**- DAYS_EMPLOYED**: Ngày bắt đầu làm việc (Số ngày tính ngược từ ngày hiện tại, 0 là ngày hiện tại, số dương là người đó hiện đang thất nghiệp)

**- FLAG_MOBIL**: Có điện thoại di động không

**- FLAG_WORK_PHONE**: Có điện thoại làm việc không

**- FLAG_PHONE**: Có điện thoại không

**- FLAG_EMAIL**: Có email không

**- OCCUPATION_TYPE**: Nghề nghiệp

**- CNT_FAM_MEMBERS**: Số lượng thành viên trong gia đình

In [8]:
df = pd.read_csv('./data-mining-project/data.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


#**KHAI PHÁ DỮ LIỆU**

#**TIỀN XỬ LÝ DỮ LIỆU**

#**HUẤN LUYỆN**

#**ĐÁNH GIÁ**

#**TINH CHỈNH**

#**THỬ NGHIỆM**